In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [3]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [4]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
line = Line([y, z])

In [6]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [7]:
SE6 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f: -c*(5*c+e)/(c+e)})

In [8]:
%time adm_SE6 = SE6.find_admissible_projectivities()

CPU times: user 329 ms, sys: 114 ms, total: 443 ms
Wall time: 1.08 s


In [9]:
len(adm_SE6)

48

In [10]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)

CPU times: user 32.1 ms, sys: 52.7 ms, total: 84.7 ms
Wall time: 119 ms


In [11]:
len(simm_SE6)

24

In [12]:
G_SE6 = Group(simm_SE6)

In [13]:
G_SE6.is_group()

True

In [14]:
G_SE6.get_order()

24

In [15]:
G_SE6.is_abelian()

False

In [16]:
G_SE6.get_divisors_of_order()

[1, 2, 3, 4, 6, 8, 12, 24]

In [17]:
G_SE6.get_order_of_elements()

[1, 2, 2, 4, 4, 3, 4, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 4, 3, 2, 4, 4, 3]

In [18]:
find_conditions_for_subfamilies(SE6, adm_SE6, simm_SE6)

[]

In [19]:
perms = []
for el in G_SE6:
    perms.append(Permutation(apply_proj_to_eck(el, SE6.eckardt_points)).to_cycles())

In [20]:
PermutationGroup(perms).order()

24

In [21]:
PermutationGroup(perms).structure_description()

'S4'